In [1]:
import os
import urllib.request as req
import numpy as np
import tensorflow as tf

#set up parameter
TRAIN = 'iris_training.csv'
TRAIN_URL = 'http://download.tensorflow.org/data/iris_training.csv'
TEST = 'iris_testing.csv'
TEST_URL = 'http://download.tensorflow.org/data/iris_test.csv'
input_shape = 4
n_classes = 3

def loadfile(filename, link):
    if not os.path.exists(filename):
        raw = req.urlopen(link).read().decode('utf-8')
        with open(filename, 'w') as f:
            f.write(raw)
    data = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=filename,
      target_dtype=np.int,
      features_dtype=np.float32)
    #normalize biến dự báo theo phân phối chuẩn
    mu = np.mean(data.data, axis = 0)
    sigma = (np.std(data.data, axis=0))
    predictor = (data.data - mu) / sigma
    
    #Chuyển biến mục tiêu sang dạng onehot endcoder
#     target = np.eye(len(data.target), n_classes, dtype = np.float32)[data.target]
    target = data.target
    return {'predictor': predictor, 'target': target}

train = loadfile(TRAIN, TRAIN_URL)
test = loadfile(TEST, TEST_URL)


X = tf.placeholder(tf.float32, [None, input_shape])
y = tf.placeholder(tf.int32, [None])

Instructions for updating:
Use tf.data instead.


In [2]:
#https://stackoverflow.com/questions/46264133/weights-and-biases-not-updating-in-tensorflow
weights = {
    'l1': tf.Variable(tf.random_normal([input_shape, 10])),
    'l2': tf.Variable(tf.random_normal([10, 15])),                              
    'l3': tf.Variable(tf.random_normal([15, 20])),
    'out': tf.Variable(tf.random_normal([20, 1]))
}


biases = {
    'l1': tf.Variable(tf.random_normal([1, 10])),
    'l2': tf.Variable(tf.random_normal([1, 15])),                              
    'l3': tf.Variable(tf.random_normal([1, 20])),
    'out': tf.Variable(tf.random_normal([1, 3]))
}
                           
    
def neural_network(X):
    layer1 = tf.nn.relu(tf.add(tf.matmul(X, weights['l1']), biases['l1']))
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, weights['l2']), biases['l2']))
    layer3 = tf.nn.relu(tf.add(tf.matmul(layer2, weights['l3']), biases['l3']))
    out = tf.add(tf.matmul(layer3, weights['out']), biases['out'])
    return out

nn = neural_network(X)



match_pred = tf.equal(tf.cast(tf.argmax(nn, 1), tf.int32), y)
acc_op = tf.reduce_mean(tf.cast(match_pred, tf.float32))



import time
#Xây dựng vòng lặp fitting model trên từng batch
batch_size = 120 #Kích thước mỗi batch.
n_steps = 200 #Số lượng các lượt cập nhật dữ liệu.
print_every = 10 #Khoảng cách lượt cập nhật dữ liệu để in ra kết quả thuật toán.
learning_rate = 0.5
#Tạo hàm lấy batch tiếp theo. Khi lấy hết đến batch cuối cùng của mẫu sẽ shuffle lại mẫu.
def next_batch(X, batch_size, index = 0):
    start = index
    index += batch_size
    if index > len(X['predictor']):
        perm = np.arange(len(X['predictor']))
        np.random.shuffle(perm)
        X['predictor'] = X['predictor'][perm]
        X['target'] = X['target'][perm]
        start = 0
        index = batch_size
    end = index
    return X['predictor'][start:end], X['target'][start:end], index




cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=nn, labels=y)
loss = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate)



def debug_minimize(optimizer, loss, sess):
    from tensorflow.python.ops import variables
    from tensorflow.python.framework import ops
    # get all varibles
    var_list = (variables.trainable_variables() + ops.get_collection(ops.GraphKeys.TRAINABLE_RESOURCE_VARIABLES))
    print('variables')
    for v in var_list:
        print('  ', v.name)
    # get all gradients
    grads_and_vars = optimizer.compute_gradients(loss)
    train_op = optimizer.apply_gradients(grads_and_vars)
    idx = 0
    for step in range(n_steps):
        start_time = time.time()
        batch_x, batch_y, idx = next_batch(train, batch_size = batch_size, index = idx)
        zipped_val = sess.run(grads_and_vars, feed_dict = {X:batch_x, y:batch_y})
        loss_value = sess.run(loss, feed_dict = {X:batch_x, y:batch_y})
        print('loss_value: %s'%str(loss_value))
        for rsl, tensor in zip(zipped_val, grads_and_vars):
            print('-----------------------------------------')
#             print('name', tensor[0].name.replace('/tuple/control_dependency_1:0', '').replace('gradients/', ''))
            print('gradient', rsl[0])
#             print('value', rsl[1])
    return train_op

sess = tf.Session()
sess.run(tf.global_variables_initializer())
debug_minimize(optimizer, loss, sess)

variables
   Variable:0
   Variable_1:0
   Variable_2:0
   Variable_3:0
   Variable_4:0
   Variable_5:0
   Variable_6:0
   Variable_7:0
loss_value: 1.5386404
-----------------------------------------
gradient [[-2.1976949e-09 -1.2098361e-08 -2.6039595e-09 -2.8330207e-09
  -3.1062878e-08  1.3999352e-08 -5.5998055e-09  1.6654544e-08
   1.6826013e-08 -1.9046644e-09]
 [-2.2012643e-09  3.4443779e-08 -6.4736061e-09 -1.5037549e-10
   6.3902426e-08 -1.4571131e-08  9.6455564e-09 -6.9443762e-09
  -3.0671384e-08  1.6197815e-08]
 [-1.0542861e-08 -2.7503695e-08  1.4056584e-10 -5.3623452e-09
  -6.0688670e-08  1.9265102e-08 -7.0441586e-09  1.0759246e-08
   2.9443331e-08 -7.2982580e-09]
 [-9.1004928e-09 -2.3862681e-08  2.4203833e-11 -4.9525077e-09
  -5.9735420e-08  2.2245819e-08 -6.6118124e-09  5.1116573e-09
   2.7594556e-08 -6.3391132e-09]]
-----------------------------------------
gradient [[ 6.12036699e-10  1.60488300e-09  0.00000000e+00 -4.94443791e-11
  -3.56068008e-10  8.27975799e-10 -8.65690714

<tf.Operation 'Adam' type=NoOp>